In [ ]:
import copy

from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd

from alns import ALNS, State
from alns.criteria import HillClimbing
from alns.weight_schemes import SimpleWeights

In [ ]:
%matplotlib inline

In [ ]:
SEED = 5432

# The resource-constrained project scheduling problem

The following explanation is largely based on [this paper](https://pms2020.sciencesconf.org/300164/document).

The goal of the RCPSP is to schedule a set of project activities $V = \{ 0, 1, 2, \ldots, n \}$, such that the makespan of the project is minimised.
Each activity $i \in V$ has a duration $d_i \in \mathbb{N}$.
Precedence constraints impose that an activity $i \in V$ can only start after all its predecessor activities have been completed.
The precedence constraints are given by a set of edges $E \subset V \times V$, where $(i, j) \in E$ means that $i$ must be completed before $j$ can commence.
Resource constraints, on the other hand, impose that an activity can only be scheduled if sufficient resources are available.
There are $K = \{ 1, 2, \ldots, m \}$ renewable resources available, with $R_k$ indicating the availability of resource $k$.
Each activity $i \in V$ requires $r_{ik}$ units of resource $k$.

A solution to the RCPSP is a schedule of activities $S = \{ S_0, S_1, \ldots, S_n \}$, where $S_i$ is the starting time of activity $i$.
The project starts at time $S_0 = 0$, and completes at $S_n$, where activities $0$ and $n$ are dummy activities that represent the start and completion of the project, respectively.
Define by $A(S, t) = \{ i \in V \mid S_i \le t \text{ and } S_i + d_i \ge t \}$ the set of activities in $S$ that are active at time $t$.
The RCPCP can then be formulated as the following optimisation problem:

\begin{align}
    \min         & S_n \\
    \text{s.t. } & S_i + d_i \le S_j \qquad \forall (i, j) \in E \\
                 & \sum_{i \in A(S, t)} r_{ik} \le R_k \qquad \forall t \ge 0,~\forall k \in K \\
                 & S_i \ge 0 \qquad \forall i \in V
\end{align}

----

In this notebook, we solve an instance of the RCPSP using ALNS.
In particular, we solve instance `j9041_6` of the [PSPLib](http://www.om-db.wi.tum.de/psplib/library.html) benchmark suite.
This instance consists of 90 jobs, and four resources.


In [ ]:
# 1. Notebook
# 2. README